In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Model Evaluations
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
import time
import sys
import pickle
 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [6]:
df=pd.read_csv("symbol_features.csv")
df

,modulation_type,symbol,label,magnitude,phase,real,imag
0,bpsk,(1+0j),1,1.000000,0.000000,1.000000,0.000000
1,bpsk,(-1+0j),0,1.000000,3.141593,-1.000000,0.000000
2,bpsk,(-1+0j),0,1.000000,3.141593,-1.000000,0.000000
3,bpsk,(-1+0j),0,1.000000,3.141593,-1.000000,0.000000
4,bpsk,(-1+0j),0,1.000000,3.141593,-1.000000,0.000000
...,...,...,...,...,...,...,...
795,256apsk,(-0.2888089479520107+0.8888625445008081j),0,0.934605,1.884956,-0.288809,0.888863
796,256apsk,(-0.28880894795201084-0.888862544500808j),0,0.934605,-1.884956,-0.288809,-0.888863
797,256apsk,(0.6776553913499502-0.9516338422362578j),1,1.168257,-0.951998,0.677655,-0.951634
798,256apsk,(1.0845720904002638-0.4341971670103332j),0,1.168257,-0.380799,1.084572,-0.434197


In [29]:

feature_cols = ['magnitude']
target_col = 'modulation_type'

X_train, X_test, y_train, y_test = train_test_split(df[feature_cols], df[target_col], test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)

clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy:.3f}')

# # Print the classification report
# print(classification_report(y_test, y_pred))

# # Print the confusion matrix
# print(confusion_matrix(y_test, y_pred))

In [26]:
clf.score(X_test,y_test)

0.88125

In [ ]:
y_pred = clf.predict(X_test)
y_pred

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
df_noise=pd.read_csv("symbol_features_noise.csv")
df_noise

In [ ]:
feature_cols = ['magnitude']
target_col = 'modulation_type'

X_train, X_test, y_train, y_test = train_test_split(df_noise[feature_cols], df_noise[target_col], test_size=0.2, random_state=42)

clf2 = RandomForestClassifier(n_estimators=100, random_state=42)

clf2.fit(X_train, y_train)

In [ ]:
clf2.score(X_test,y_test)

In [ ]:
y_pred = clf.predict(X_test)
y_pred

In [ ]:
print(classification_report(y_pred,y_test))

In [33]:
def weighted_score(model, X_test, y_test, weight_for_latency = 0.2, weight_for_model_size=0.1):
    try:
        accuracy = model.score(X_test, y_test)        
        start_time = time.time()
        _ = model.predict(X_test)
        end_time = time.time()
        latency = (end_time - start_time) / len(X_test)

        model_pickle = pickle.dumps(clf)
        model_size = len(model_pickle)

        max_latency = 0.02
        max_model_size = 2e6

        normalized_latency = latency/max_latency
        normalized_size = model_size/max_model_size

        score = ( accuracy) + (weight_for_latency * normalized_latency) + (weight_for_model_size * normalized_size)
        
        result_string = (
            f"Accuracy: {accuracy:.3f}\n"
            f"Normalized Latency: {normalized_latency:.3f} and Latency: {latency}\n"
            f"Normalized Size: {normalized_size:.3f} and Size: {model_size} in Bytes\n"
            f"Final Power-Efficient Score: {score:.3f}")

        return result_string

    
    except Exception as e:
        print(f"Error calculating power-efficient score: {e}")
        return None

In [34]:
print(weighted_score(clf, X_test, y_test))

Accuracy: 1.000
Normalized Latency: 0.004 and Latency: 8.090734481811524e-05
Normalized Size: 0.313 and Size: 626744 in Bytes
Final Power-Efficient Score: 0.732
